In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/ifajcik/PycharmProjects/efficientQA'

In [2]:
! ls | grep 'NQ' 

NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_abstractive_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_fused.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_gen_reranked_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_preprocessed_queryenc_wikipassages_NQ_open_val_<class 'transformers.tokenization_roberta.RobertaTokenizer'>.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_ranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl
NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reranked_outputs.jsonl_fuseindecoder_preprocessed_for_C25_t5-base_L512.json
NQ-open.efficientqa.dev.1.1.no-annotations.jsonlfused_w_abs.json


In [3]:
import json


extractive_reader_output_f = "NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_reader_outputs.json"
genreranked_reader_predictions_f  ="NQ-open.efficientqa.dev.1.1.no-annotations.jsonl_gen_reranked_reader_outputs.json"


gt_file = "efficientqa_eval/NQ-open.efficientqa.dev.1.1.jsonl"

with open(genreranked_reader_predictions_f, encoding="utf8") as f:
    genreranked_reader_predictions =  json.load(f)
with open(extractive_reader_output_f, encoding="utf8") as f:
    extractive_reader_predictions = json.load(f)


inference_type="hardmax" # hardmax / marginal

MAX_ANS = 50
    
import jsonlines
correct = {x['question']:x['answer'] for x in list(jsonlines.open(gt_file))}

In [4]:
print(f"Maximum number of fused answers: {len(extractive_reader_predictions['0']['answers'])}")

Maximum number of fused answers: 50


### Generative re-ranking only  filters answer of whitespacespan shorter than 10, remove this when martin fixes answer lengts

In [5]:
import copy

genreranked_reader_predictions_trunc=dict()
for k,v in genreranked_reader_predictions.items():
    g_p = copy.deepcopy(v)
    g_p ['reader_scores'] = g_p ['reader_scores'][:MAX_ANS]
    g_p ['answers'] = g_p ['answers'][:MAX_ANS]
    
    genreranked_reader_predictions_trunc[k]= g_p
genreranked_reader_predictions = genreranked_reader_predictions_trunc

In [6]:
import copy
import math

genreranked_reader_predictions_corrected = dict()
# Deduplicate
for k,v, in genreranked_reader_predictions.items():
    g_p = copy.deepcopy(v)
    scores , answers= [], []
    for a,s in zip(v['answers'], v["reader_scores"]):
            if not a in answers: 
                scores.append(s)
                answers.append(a)
            
    g_p ['reader_scores'] = scores
    g_p ['answers'] = answers
    genreranked_reader_predictions_corrected[k] = g_p
genreranked_reader_predictions = genreranked_reader_predictions_corrected
    
# Deduplicate and keep only ones that are in generative
extractive_reader_predictions_corrected = dict()
for k,v in extractive_reader_predictions.items():
    e_p = copy.deepcopy(v)
    # Dirty heuristic filtering, TODO: remove when Martin fixes answer length filtering
    scores , answers= [], []
    for a,s in zip(v['answers'], v["reader_scores"]):
        if a in genreranked_reader_predictions[k]['answers']:
            if a in answers:
                if inference_type == "marginal":
                    idx = answers.index(a)
                    scores[idx]+= math.exp(s) # add score of this answer span
            else:
                scores.append(math.exp(s))
                answers.append(a)  
    e_p ['reader_scores'] = [math.log(s) for s in scores]
    e_p ['answers'] = answers
    extractive_reader_predictions_corrected[k]= e_p
    
extractive_reader_predictions = extractive_reader_predictions_corrected

In [7]:
# make sure all answers are the same
for i, (e_p, g_p) in enumerate(zip(extractive_reader_predictions.values(), genreranked_reader_predictions.values())):
    if not len(g_p["answers"]) == len(e_p["answers"]):
        print(i)
        print(g_p["answers"])
        print(g_p["reader_scores"])
        
        print(e_p["answers"])
    assert len(g_p["answers"]) == len(e_p["answers"])
    for ag, ae in zip(g_p["answers"],e_p["answers"]):
            if not ag == ae:
                print(g_p["answers"],e_p["answers"])
            assert ag == ae

Get only predictions, which contains correct answer in top-k

In [8]:
import fit_qa.scripts.common.evaluate_predictions as eval_utils
from fit_qa.scripts.common.evaluate_predictions import exact_match_score
labels,X,Y = [], [], [] 

for e_p, g_p in zip(extractive_reader_predictions.values(), genreranked_reader_predictions.values()):
    assert e_p['raw_question'] == g_p['raw_question']
    correct_answers = correct[e_p['raw_question']]
    gt_rank = -1 
    for i, a in enumerate(g_p['answers']):
        if eval_utils.metric_max_over_ground_truths(
            metric_fn=exact_match_score, prediction=a, ground_truths=correct_answers):
            gt_rank = i
            break
    if gt_rank <0:
        continue
    labels.append(gt_rank)
    
    if len(g_p['reader_scores'])<MAX_ANS:
        g_p['reader_scores'] += [-10_000]*(MAX_ANS - len(g_p['reader_scores']))
    if len(e_p['reader_scores'])<MAX_ANS:
        e_p['reader_scores'] += [-10_000]*(MAX_ANS - len(e_p['reader_scores']))
    
    X.append(e_p['reader_scores'])
    Y.append(g_p['reader_scores'])
 

In [9]:
import fit_qa.scripts.common.evaluate_predictions as eval_utils
from fit_qa.scripts.common.evaluate_predictions import exact_match_score

X_test,Y_test = [], [] 

for e_p, g_p in zip(extractive_reader_predictions.values(), genreranked_reader_predictions.values()):
    assert e_p['raw_question'] == g_p['raw_question']
    
    if len(g_p['reader_scores'])<MAX_ANS:
        g_p['reader_scores'] += [-10_000]*(MAX_ANS - len(g_p['reader_scores']))
    if len(e_p['reader_scores'])<MAX_ANS:
        e_p['reader_scores'] += [-10_000]*(MAX_ANS - len(e_p['reader_scores']))
    
    X_test.append(e_p['reader_scores'])
    Y_test.append(g_p['reader_scores'])

In [10]:
import torch
labels = torch.LongTensor(labels)
X = torch.FloatTensor(X)
Y = torch.FloatTensor(Y)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)
X, Y, labels[:10]

(tensor([[-4.7501e-04, -2.8762e+01, -2.9989e+01,  ..., -5.5894e+01,
          -5.5960e+01, -1.0000e+04],
         [-2.3366e-03, -1.6164e+01, -1.7683e+01,  ..., -4.4904e+01,
          -1.0000e+04, -1.0000e+04],
         [-1.7982e+00, -6.9578e+00, -8.3420e+00,  ..., -1.0000e+04,
          -1.0000e+04, -1.0000e+04],
         ...,
         [-2.0917e+00, -1.7857e+01, -2.1356e+01,  ..., -1.0000e+04,
          -1.0000e+04, -1.0000e+04],
         [-1.2662e-02, -1.9288e+01, -2.2638e+01,  ..., -1.0000e+04,
          -1.0000e+04, -1.0000e+04],
         [-4.8160e-05, -2.4338e+01, -2.5361e+01,  ..., -1.0000e+04,
          -1.0000e+04, -1.0000e+04]]),
 tensor([[-1.9833e-01, -4.0924e+00, -1.1933e+01,  ..., -2.1963e+01,
          -2.3005e+01, -1.0000e+04],
         [-1.1419e-01, -6.9511e+00, -9.5937e+00,  ..., -1.2082e+01,
          -1.0000e+04, -1.0000e+04],
         [-4.7947e-01, -2.2238e+00, -1.0346e+01,  ..., -1.0000e+04,
          -1.0000e+04, -1.0000e+04],
         ...,
         [-2.9003e+00, -5

In [11]:
X.shape, Y.shape, labels.shape

(torch.Size([1224, 50]), torch.Size([1224, 50]), torch.Size([1224]))

In [12]:
X_test.shape, Y_test.shape

(torch.Size([1800, 50]), torch.Size([1800, 50]))

In [13]:
import torch.nn.functional as F
F.cross_entropy(X+Y, labels)

tensor(4.9409)

In [14]:
torch.Tensor([-5]).abs()

tensor([5.])

In [15]:
import torch
class ConstrainedLR(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.λ = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        self.α = torch.nn.Parameter(torch.randn((1), requires_grad=True))
        
    def forward(self,X,Y, mask=None):
        r =X*self.α +  Y*self.λ 
        # I also tried
        # torch.exp(X)*self.α + torch.exp(Y)*self.λ 
        # torch.exp(X*self.α) + torch.exp(Y*self.λ )

        if mask is not None:
            r[mask]=-float("inf")
        return r

In [16]:
def evaluate(preds, gts):
    assert len(preds) == len(gts)
    hits = 0
    for q, a in preds.items():
        gt_answers = gts[q]
        hits+= int(eval_utils.metric_max_over_ground_truths(
            metric_fn=exact_match_score, prediction=a, ground_truths=gt_answers))
    acc = hits*100./len(preds)
    return acc

In [17]:
def evaluate_model(S):
    fused_predictions = dict()
    assert S.shape[0] == len(extractive_reader_predictions)
    for i, (e_p, g_p) in enumerate(zip(extractive_reader_predictions.values(), genreranked_reader_predictions.values())):
        assert e_p['raw_question'] == g_p['raw_question']
        fused_scores= S[i].tolist()
        def argmax(l):
            f = lambda i: l[i]
            return max(range(len(l)), key=f)
        if len( e_p["answers"])<=argmax(fused_scores):
            print(fused_scores)
            print(i)
            print(e_p["answers"])
        fused_predictions[e_p["raw_question"]] =  e_p["answers"][argmax(fused_scores)]
    return evaluate(fused_predictions,correct)

In [18]:
def run_training():
    import torch.nn.functional as F
    from tqdm import tqdm
    from transformers import get_linear_schedule_with_warmup

    training_mask = (X+Y)<-9999
    test_mask = (X_test+Y_test)<-9999
    STEPS=100
    model =  ConstrainedLR()
    model = model.train()
    opt = torch.optim.SGD(model.parameters(), lr=0.3)
    scheduler = get_linear_schedule_with_warmup(
                    opt,
                    num_warmup_steps=10,
                    num_training_steps=STEPS
                )

    best_acc= 0
    best_r = None
    iterator = tqdm(range(STEPS))
    for i in iterator:
        logits = model(X, Y, training_mask)
        l_list = F.cross_entropy(logits, labels, reduction='none')
        l = l_list.mean()
        l.backward()
        opt.step()
        scheduler.step()
        opt.zero_grad()
        if i % 1 == 0:
            r = { k: v.item() for k,v in dict(model.named_parameters()).items()}
            model.eval()
            acc = evaluate_model(model(X_test,Y_test, test_mask))
            model.train()
            if acc>best_acc:
                iterator.set_description(str(acc))
                best_acc = acc
                best_r = r
    return best_acc, best_r
total_best_acc, total_best_r = 0.,None
for _ in range(20):
    acc,r = run_training()
    if acc>total_best_acc:
        print(r)
        total_best_acc=acc
        total_best_r=r

42.611111111111114:   2%|▏         | 2/100 [00:00<00:05, 16.44it/s]

{'λ': 0.6657118201255798, 'α': 0.08693467080593109}


46.44444444444444:   2%|▏         | 2/100 [00:00<00:05, 16.44it/s]

{'λ': 0.4853821098804474, 'α': 0.04384877532720566}


45.77777777777778:   0%|          | 0/100 [00:00<?, ?it/s] 

{'λ': 0.5088504552841187, 'α': 0.053242623805999756}


47.111111111111114: 100%|██████████| 100/100 [00:06<00:00, 14.48it/s]


e.g.   
```
Best accuracy: 47.166666666666664
Best parameters:
{'λ': 0.4845413267612457, 'α': 0.05110572651028633}
```

In [19]:
print(f"Best accuracy: {total_best_acc}")
print(f"Best parameters:\n{total_best_r}")

Best accuracy: 47.166666666666664
Best parameters:
{'λ': 0.5088504552841187, 'α': 0.053242623805999756}


In [20]:
model =  ConstrainedLR()
model = model.eval()
test_mask = (X_test+Y_test)<-9999

In [21]:
best_r_sd = { k: torch.FloatTensor([v]) for k,v in total_best_r.items()}
model.load_state_dict(best_r_sd)


<All keys matched successfully>

In [22]:
# Use only extractive reader
model.λ[0] = 0.
evaluate_model(model(X_test,Y_test, test_mask))

42.611111111111114

In [23]:
best_r_sd = { k: torch.FloatTensor([v]) for k,v in total_best_r.items()}
model.load_state_dict(best_r_sd)

<All keys matched successfully>

In [24]:
# Use only generative reader (scoring extractive predictions!)
model.α[0] = 0.
evaluate_model(model(X_test,Y_test, test_mask))

45.22222222222222